In [1]:
from ultralytics import YOLO
import helpers
from helpers import *

import os 

<font size=6> Introduction <br>
<font size = 4>This notebook is very similar to the API-model folder but the only difference here is that the model used is YOLO11 which I trained by myself. I downloaded a dataset from Roboflow consisting of 500 images. Th training was utilizing the free remote GPU provided by Google Collab. I trained my model on the dataset and downloaded the best version of the model locally. The model was trained for 30 epochs and attained relatively high mAP50 score of 0.976 and 0.63 for mAP50-95, given the small training data size.


In [ ]:
model = YOLO('best.pt')
results = model('sample_image.png')

object_count = len(results[0].boxes)
print(f" The number of barnacles is: {object_count}")


image 1/1 c:\Users\arses\Barnacles\sample_image.png: 640x640 300 barnacles, 282.0ms
Speed: 5.7ms preprocess, 282.0ms inference, 11.6ms postprocess per image at shape (1, 3, 640, 640)
300


<font size = 4> As you can see, the number of barnacles detected in from the provided sample is 300, which is obviously far less than the real number of barnacles. This is again due to the fact that the dataset on which the model was trained only contained a few barnacles per image. To combat this problem, we will follow the same procedure that we did with the api-model and break the image down into much smaller chunks and calculate the number of barnacles seperately. 

In [3]:
output_directory  = "C:/Users/arses/Desktop/Cropped_images"
image_path = "C:/Users/arses/Downloads/sample_image.png"

try : 
    os.mkdir(output_directory)
    print(f"Directory '{output_directory}' created successfully.")

#Already created this file
except FileExistsError: 
    print(f"Directory '{output_directory}' already exists.")

#Check access and permissions settings 
except PermissionError: 
    print(f"Permission denied: Unable to create '{output_directory}.")

#Any other type of errors 
except Exception as e: 
    print(f"An error occured:{e}")

Directory 'C:/Users/arses/Desktop/Cropped_images' already exists.


Crop the image into 256 smaller images to later feed to the model 

In [4]:
crop_image_into_tiles(image_path,output_directory)

Image cropped successfully


<font size = 5>Running the loop for the counting the number of barnacles

In [ ]:
number_of_barnacles= 0
number_of_images= directory_size(output_directory)
for i in range(number_of_images):
    
    result = model(f"{output_directory}/tile_{i}.png", verbose= False)
    count = len(result[0].boxes)
    number_of_barnacles+= count

print(f"The number of barnacles present in the image is: {number_of_barnacles}")


Number of files in 'C:/Users/arses/Desktop/Cropped_images': 289



image 1/1 C:\Users\arses\Desktop\Cropped_images\tile_0.png: 640x640 5 barnacles, 235.8ms
Speed: 4.5ms preprocess, 235.8ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\arses\Desktop\Cropped_images\tile_1.png: 640x640 2 barnacles, 265.2ms
Speed: 6.9ms preprocess, 265.2ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\arses\Desktop\Cropped_images\tile_2.png: 640x640 8 barnacles, 230.9ms
Speed: 4.6ms preprocess, 230.9ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\arses\Desktop\Cropped_images\tile_3.png: 640x640 2 barnacles, 203.4ms
Speed: 5.7ms preprocess, 203.4ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\arses\Desktop\Cropped_images\tile_4.png: 640x640 4 barnacles, 189.4ms
Speed: 4.5ms preprocess, 189.4ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\arses\Desktop\Cropped_images\tile_5.png: 

<font size = 5>Removing the saved images from the directory 

In [ ]:
remove_files_from_directory(output_directory)